# ETAPA 1 - Preparação dos Dados

### #=====Vizualização dos Dados======#

Nesta etapa, foi realizada a leitura de todos os arquivos CSV da base de dados do Olist, localizados na pasta data/raw/.
Utilizei um dicionário (file_map) para mapear nomes intuitivos aos nomes dos arquivos. Em seguida, carreguei os dados com pandas.read_csv() e armazenamos os DataFrames em um dicionário (dfs) para facilitar o acesso e manipulação. Por fim, foi exibida as primeiras linhas de cada tabela para verificar se os dados foram carregados corretamente.

In [ ]:
import pandas as pd
import os

#===Definindo o caminho dos arquivos brutos===#
raw_path = '../data/raw/'

#===Mapeando os arquivos em um dicionário para facilitar o carregamento===#
file_map = {
    "orders": "olist_orders_dataset.csv",
    "customers": "olist_customers_dataset.csv",
    "products": "olist_products_dataset.csv",
    "sellers": "olist_sellers_dataset.csv",
    "order_items": "olist_order_items_dataset.csv",
    "order_reviews": "olist_order_reviews_dataset.csv",
    "order_payments": "olist_order_payments_dataset.csv",
    "geolocation": "olist_geolocation_dataset.csv",
    "product_translation": "product_category_name_translation.csv"
}

#===Carregando todos os arquivos em dataframes===#
dfs = {name: pd.read_csv(os.path.join(raw_path, filename)) for name, filename in file_map.items()}

#===Exibindo os primeiros registros de cada dataset===#
for name, df in dfs.items():
    print(f"\n{name.upper()}:\n")
    display(df.head())


### #======Limpeza e Normalização dos Dados======#

Nesta etapa, apliquei duas funções auxiliares para padronizar e limpar todos os DataFrames carregados:

- normalize_columns(): transforma os nomes das colunas para minúsculo, substitui espaços e hífens por underscores (_), deixando o padrão mais limpo e consistente.

- clean_df(): remove linhas duplicadas, linhas e colunas totalmente vazias.

Após aplicar essas funções a todos os DataFrames, salvei os arquivos limpos na pasta data/processed/, organizando o fluxo de dados para etapas futuras da análise.

In [ ]:
#===Função que normaliza nomes de colunas===#
def normalize_columns(df):
    return df.rename(columns=lambda x: x.strip().lower().replace(" ", "_").replace("-", "_"))

#===Função de limpeza básica===#
def clean_df(df):
    df = df.drop_duplicates()           #->Remove duplicatas
    df = df.dropna(how='all')           #->Remove linhas totalmente vazias
    df = df.dropna(axis=1, how='all')   #->Remove colunas totalmente vazias
    return df

#===Aplicando limpeza e normalização em todos os dataframes===#
for name in dfs:
    dfs[name] = normalize_columns(dfs[name])
    dfs[name] = clean_df(dfs[name])

#===Correções específicas para colunas com erros de digitação===#
dfs["products"].rename(columns={
    "product_name_lenght": "product_name_length",
    "product_description_lenght": "product_description_length"
}, inplace=True)

#===Salvando os dados limpos em CSVs===#
processed_path = "../data/processed/"
if os.path.exists(processed_path) and not os.path.isdir(processed_path):
    os.remove(processed_path)
os.makedirs(processed_path, exist_ok=True)

for name, df in dfs.items():
    df.to_csv(os.path.join(processed_path, f"{name}.csv"), index=False)


### #======Criação do banco de dados relacional======#

-Defini as tabelas com chaves primárias e estrangeiras, respeitando as relações entre os dados (ex: order_items referenciando orders, products e sellers).

-Inseri todos os dados do dicionário dfs diretamente nas tabelas do banco, utilizando pandas.to_sql() com if_exists="append".

In [ ]:
import sqlite3
import os

#===Caminho para salvar o banco SQLite===#
processed_path = "../data/processed"
db_path = os.path.join(processed_path, "olist_ecommerce.db")

#===Conexão com o banco===#
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

#===Criação das tabelas com chaves primárias e estrangeiras===#
cursor.executescript("""
DROP TABLE IF EXISTS customers;
DROP TABLE IF EXISTS sellers;
DROP TABLE IF EXISTS products;
DROP TABLE IF EXISTS orders;
DROP TABLE IF EXISTS order_items;
DROP TABLE IF EXISTS order_reviews;
DROP TABLE IF EXISTS order_payments;
DROP TABLE IF EXISTS geolocation;
DROP TABLE IF EXISTS product_translation;

CREATE TABLE customers (
    customer_id TEXT PRIMARY KEY,
    customer_unique_id TEXT,
    customer_zip_code_prefix INTEGER,
    customer_city TEXT,
    customer_state TEXT
);

CREATE TABLE sellers (
    seller_id TEXT PRIMARY KEY,
    seller_zip_code_prefix INTEGER,
    seller_city TEXT,
    seller_state TEXT
);

CREATE TABLE products (
    product_id TEXT PRIMARY KEY,
    product_category_name TEXT,
    product_name_length REAL,
    product_description_length REAL,
    product_photos_qty REAL,
    product_weight_g REAL,
    product_length_cm REAL,
    product_height_cm REAL,
    product_width_cm REAL
);

CREATE TABLE orders (
    order_id TEXT PRIMARY KEY,
    customer_id TEXT,
    order_status TEXT,
    order_purchase_timestamp TEXT,
    order_approved_at TEXT,
    order_delivered_carrier_date TEXT,
    order_delivered_customer_date TEXT,
    order_estimated_delivery_date TEXT,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);

CREATE TABLE order_items (
    order_id TEXT,
    order_item_id INTEGER,
    product_id TEXT,
    seller_id TEXT,
    shipping_limit_date TEXT,
    price REAL,
    freight_value REAL,
    PRIMARY KEY (order_id, order_item_id),
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id),
    FOREIGN KEY (seller_id) REFERENCES sellers(seller_id)
);

CREATE TABLE order_reviews (
    review_id TEXT PRIMARY KEY,
    order_id TEXT,
    review_score INTEGER,
    review_comment_title TEXT,
    review_comment_message TEXT,
    review_creation_date TEXT,
    review_answer_timestamp TEXT,
    FOREIGN KEY (order_id) REFERENCES orders(order_id)
);

CREATE TABLE order_payments (
    order_id TEXT,
    payment_sequential INTEGER,
    payment_type TEXT,
    payment_installments INTEGER,
    payment_value REAL,
    FOREIGN KEY (order_id) REFERENCES orders(order_id)
);

CREATE TABLE geolocation (
    geolocation_zip_code_prefix INTEGER,
    geolocation_lat REAL,
    geolocation_lng REAL,
    geolocation_city TEXT,
    geolocation_state TEXT
);

CREATE TABLE product_translation (
    product_category_name TEXT PRIMARY KEY,
    product_category_name_english TEXT
);
""")

#===Removendo duplicatas com base nas chaves primárias antes da inserção===#
dfs["customers"].drop_duplicates(subset=["customer_id"], inplace=True)
dfs["sellers"].drop_duplicates(subset=["seller_id"], inplace=True)
dfs["products"].drop_duplicates(subset=["product_id"], inplace=True)
dfs["orders"].drop_duplicates(subset=["order_id"], inplace=True)
dfs["order_reviews"].drop_duplicates(subset=["review_id"], inplace=True)
dfs["order_items"].drop_duplicates(subset=["order_id", "order_item_id"], inplace=True)
dfs["product_translation"].drop_duplicates(subset=["product_category_name"], inplace=True)

# Inserção dos dados em cada tabela
for name, df in dfs.items():
    df.to_sql(name, conn, if_exists="append", index=False)

conn.commit()
conn.close()


### #======Verificação do Banco de Dados======#

Listei todas as tabelas criadas no SQLite para garantir que foram geradas corretamente.

Também executei um JOIN entre orders, customers e order_payments para validar os relacionamentos e conferir se os dados estão integrados como esperado.

In [ ]:
#===Conectando ao banco SQLite===#
import sqlite3

conn = sqlite3.connect("../data/processed/olist_ecommerce.db")

# Conferindo se as tabelas foram criadas no banco
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
tables

# Validando relacionamento com SQL JOIN
query = """
SELECT 
    o.order_id,
    o.order_purchase_timestamp,
    c.customer_state,
    p.payment_value
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN order_payments p ON o.order_id = p.order_id
LIMIT 5;
"""

pd.read_sql(query, conn)


# ETAPA 2 - Análise Exploratória de Dados

### a) Qual o volume de pedidos por mês? Existe sazonalidade nas vendas?

In [ ]:
#===Conectando ao banco SQLite onde estão os dados organizados===#
import sqlite3
import pandas as pd

conn = sqlite3.connect("../data/processed/olist_ecommerce.db")  #->Abre conexão com o banco de dados

#===Consulta SQL para contar quantos pedidos foram feitos por mês===#
query = """
SELECT
    strftime('%Y-%m', order_purchase_timestamp) AS order_month, 
    COUNT(order_id) AS total_orders                             
FROM orders
GROUP BY order_month
ORDER BY order_month;
"""

#===Lê o resultado da query e transforma em DataFrame===#
df_orders_by_month = pd.read_sql(query, conn)

#===Visualizando os primeiros resultados===#
df_orders_by_month.head()


Com a query acima, conseguimos agrupar os pedidos por mês utilizando a função `strftime('%Y-%m', ...)`.  
Esse agrupamento nos permitirá identificar flutuações mensais e possíveis padrões sazonais nas vendas.


In [ ]:
#===Importando bibliotecas para visualização===#
import matplotlib.pyplot as plt
import seaborn as sns

#===Criando gráfico de linha para visualizar a evolução dos pedidos mês a mês===#
plt.figure(figsize=(14,6))
sns.lineplot(data=df_orders_by_month, x="order_month", y="total_orders", marker="o")  #->Gráfico de linha com marcação nos pontos
plt.xticks(rotation=45)                                                                #->Rotaciona os rótulos do eixo x
plt.title("Volume de Pedidos por Mês")                                                 #->Título do gráfico
plt.xlabel("Mês")
plt.ylabel("Número de Pedidos")
plt.tight_layout()
plt.show()
